In [1]:
from torchvision import models
import torch.utils.data.dataset
import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer
import torch.optim as optimize
import image_loader as il
import properties as pr
import copy
from torch.multiprocessing import Pool, Process, set_start_method

#torch.multiprocessing.set_start_method('spawn', force=True)
device = 'cpu'
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
                               
print(torch.cuda.is_available())
print( torch.cuda.get_device_name(0))
"""
r = il.map_inputs_labels(pr.data_inputs_path, pr.data_labels_path)

train, validate = il.split_set(r, pr.train_size, pr.test_size)

il.prepare_data(train, validate)

for i in train:
    il.save_data(pr.tensor_train_path, i)
for j in validate:
    il.save_data(pr.tensor_validate_path, j)
"""

train_tensor = il.load_tensors(pr.tensor_train_path)
validate_tensor = il.load_tensors(pr.tensor_validate_path)

"""
train_tensor = list(map(il.apply_resize, train_tensor))
validate_tensor = list(map(il.apply_resize, validate_tensor))

for i in train_tensor:
    il.save_data(pr.tensor_train_path, i)
for j in validate_tensor:
    il.save_data(pr.tensor_validate_path, j)
"""

train_tensor = list(map(il.remove_empty_mask, train_tensor))
validate_tensor = list(map(il.remove_empty_mask, validate_tensor))

/home/nikita/anaconda3/envs/ml-diplom/lib/python3.7/site-packages/torchvision/transforms/transforms.py:210: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


True
GeForce GTX 1060 with Max-Q Design


In [8]:
def get_model_parameters_to_update(m:models.AlexNet) -> list:
    params = []
    for name, param in m.named_parameters():
        # maybe here better check require gradient back propagation
        # and then update
        # print("name={}".format(name))
        params.append(param)
    return params

def set_model_last_layer(m:models.AlexNet) -> models.AlexNet:
    num_features = m.classifier[6].in_features
    m.classifier[6] = nn.Linear(num_features, pr.labels_number) # here 2 for each label 
    return m

def train_model_single_epoch(m:models.AlexNet, data_loader:torch.utils.data.DataLoader, criterion:nn.CrossEntropyLoss, 
                             optimizer:Optimizer) -> models.AlexNet:
    m.train()
    for inputs, labels in data_loader:
        inputs = inputs.to(device)
        labels = labels.type(torch.FloatTensor).to(device)
        optimizer.zero_grad()
        
        model_result = m(inputs)
        #print(model_result)
        #print("-" * 20)
        #print(labels)
        loss = criterion(model_result, labels)
        print(loss)
        loss.backward()
        optimizer.step()
    return m

def validate_model_single_epoch(m:models.AlexNet, data_loader:torch.utils.data.DataLoader) -> float:
    confusion_matrix = torch\
        .tensor([[[0 for _ in range(2)] for _ in range(2)] for _ in range(pr.labels_number)])\
        .to(device)
    
    m.eval()
    for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            # trust result
            for t in labels:
                for idx, v in enumerate(t):
                    confusion_matrix[idx][v.item()][v.item()] += 1

            model_result = m(inputs)
            # all vectors size 5
            for i, t in enumerate(model_result):
                # all classes
                for j, v in enumerate(t):
                    dst_0 = abs(v.item())
                    dst_1 = abs(1 - v.item())
                    trust = labels[i][j].item()
                    model_answer = 1 if dst_1 < dst_0 else 0 
                    confusion_matrix[j][model_answer][trust] += 1                    
    
    print([calculate_f_measure(i) for i in confusion_matrix])
    return sum([calculate_f_measure(i) for i in confusion_matrix]) / pr.labels_number

def calculate_f_measure(conf_matrix:torch.Tensor) -> float:
    precision_w = 0
    recall_w = 0
    sum_all = conf_matrix.sum().item()
    
    for i, t in enumerate(conf_matrix):
        c = 0
        p = 0
        for j, _ in enumerate(t):
            c += conf_matrix[i][j].item()
            p += conf_matrix[j][i].item()
            if p == 0:
                precision_w += 0
            else:            
                precision_w += ((conf_matrix[i][i].item() * c) / p) /  sum_all
        recall_w += conf_matrix[i][i].item() / sum_all
    return (2 * (precision_w * recall_w)) / (precision_w + recall_w)
 

def train_model(m:models.AlexNet, train_loader:torch.utils.data.DataLoader, validate_loader:torch.utils.data.DataLoader,
                criterion:nn.CrossEntropyLoss, 
                optimizer:Optimizer, epochs:int) -> models.AlexNet:
     best_model_state = copy.deepcopy(model.state_dict())
     best_acc = 0.0
     for epoch in range(epochs):
          print("Epoch: {}/{}".format(epoch, epochs))
          print("-" * 10)
          m = train_model_single_epoch(m, train_loader, criterion, optimizer)
          accurency = validate_model_single_epoch(m, validate_loader)
          print("acc:{}".format(accurency))
          if accurency > best_acc:
               best_acc = accurency
               best_model_state = copy.deepcopy(m.state_dict())
             
     best_model = m.load_state_dict(best_model_state)
     print(best_acc)
     return best_model
    
    

In [9]:
### EXECUTE ONLY ONCE!!!!!!!!!!!
model = models.alexnet(pretrained=True, progress=False)
model = set_model_last_layer(model)

parameters = get_model_parameters_to_update(model)
data_train = torch.utils.data.DataLoader(il.CustomDataset(train_tensor), batch_size=4, shuffle=True, num_workers=4)
data_validate = torch.utils.data.DataLoader(il.CustomDataset(validate_tensor), batch_size=4, shuffle=True, num_workers=4)
crit = nn.SmoothL1Loss()
optim = optimize.SGD(parameters, lr=0.001, momentum=0.9)


In [10]:
train_model(model, data_train, data_validate, crit, optim, 10)

Epoch: 0/10
----------
tensor(0.6407, grad_fn=<SmoothL1LossBackward>)
tensor(0.3992, grad_fn=<SmoothL1LossBackward>)
tensor(0.1331, grad_fn=<SmoothL1LossBackward>)
tensor(0.1047, grad_fn=<SmoothL1LossBackward>)
tensor(0.1369, grad_fn=<SmoothL1LossBackward>)
tensor(0.2340, grad_fn=<SmoothL1LossBackward>)
tensor(0.1323, grad_fn=<SmoothL1LossBackward>)
tensor(0.2564, grad_fn=<SmoothL1LossBackward>)
tensor(0.2272, grad_fn=<SmoothL1LossBackward>)
tensor(0.2023, grad_fn=<SmoothL1LossBackward>)
tensor(0.1083, grad_fn=<SmoothL1LossBackward>)
tensor(0.1650, grad_fn=<SmoothL1LossBackward>)
tensor(0.2090, grad_fn=<SmoothL1LossBackward>)
tensor(0.1944, grad_fn=<SmoothL1LossBackward>)
tensor(0.1606, grad_fn=<SmoothL1LossBackward>)
tensor(0.0920, grad_fn=<SmoothL1LossBackward>)
tensor(0.0836, grad_fn=<SmoothL1LossBackward>)
tensor(0.1067, grad_fn=<SmoothL1LossBackward>)
tensor(0.1286, grad_fn=<SmoothL1LossBackward>)
tensor(0.0456, grad_fn=<SmoothL1LossBackward>)
tensor(0.1420, grad_fn=<SmoothL1LossB

KeyboardInterrupt: 